In [121]:
import recordlinkage as rl
import pandas as pd

In [122]:
schema_path = "./csv/schema_final.csv"

In [123]:
schema = pd.read_csv(schema_path, index_col='id')

# Preprocessamento

In [124]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [125]:
schema = schema.iloc[:10]
schema_cleaned = schema
print(schema)

                                        name country market cap founded year  \
id                                                                             
0                         Berkshire Hathaway     NaN        NaN       1939.0   
1                                       ICBC     NaN        NaN       1984.0   
2   Saudi Arabian Oil Company (Saudi Aramco)     NaN        NaN       1933.0   
3                             JPMorgan Chase     NaN        NaN       2000.0   
4                    China Construction Bank     NaN        NaN       2014.0   
5                                     Amazon     NaN        NaN       1994.0   
6                                      Apple     NaN        NaN       1976.0   
7                 Agricultural Bank of China     NaN        NaN       1979.0   
8                            Bank of America     NaN        NaN       1904.0   
9                               Toyota Motor     NaN        NaN       1937.0   

   employees                           

In [126]:
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze()
    nome = pd.Series(row["name"])
    nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=True, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
print(schema_cleaned)

TypeError: data type 'literal' not understood

# Occorrenze token per riga

In [ ]:
value_occurence(schema)

ValueError: Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional